In [1]:
#IMPORTS

import pandas as pd
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns


In [ ]:
full_analysis_df = pd.read_excel('c:/Users/u1172791/COH files/Data_2025/COMBINED_TEXT_jan-june_pre_in-gallery_post_w_SASSY_urban_rural.xlsx')
emotions_mean_by_ida = pd.read_excel('c:/Users/u1172791/COH files/Data_2025/emotion_means_by_id.xlsx')
display(emotions_mean_by_ida)

,ida,afraid_scared,angry_frustrated,anxious,guilty,happy,hopeless,indifferent,inspired,interested,numb,optimistic_hopeful,sad,stressed
0,24012401BF,1.00,0.75,1.00,0.50,0.75,0.25,0.25,1.00,1.50,0.00,1.25,0.75,0.75
1,24012401MG,0.00,0.25,0.25,0.00,0.50,0.25,0.25,0.75,2.25,0.50,1.75,0.50,0.00
2,24012401PM,0.75,1.25,1.00,1.00,1.00,0.25,0.50,1.50,2.00,0.00,2.00,1.00,1.00
3,24012601LS,0.75,1.00,0.00,0.75,1.00,0.50,0.00,1.00,2.00,1.00,2.50,1.00,1.00
4,24012601RS,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,2.00,0.00,0.50,0.25,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,24040610EC,0.00,0.00,0.25,0.00,1.50,0.00,0.50,1.00,2.25,0.00,1.75,0.25,0.50
180,24040610MC,1.00,0.75,0.50,0.00,0.50,1.00,0.75,0.25,1.50,0.00,1.00,1.00,0.50
181,24040610OW,1.50,1.25,1.25,1.00,0.75,1.50,0.50,0.75,2.25,1.00,1.00,1.25,1.00
182,24040611KS,1.25,1.50,0.50,0.25,1.50,1.50,0.25,1.50,2.50,0.25,1.50,1.00,0.25


In [3]:
# Test normality and homogeneity of variance for each emotion

# get emotion columns
emotion_cols = [col for col in emotions_mean_by_ida.columns if col != 'ida']

# Shapiro-Wilk test for normality
print("\nShapiro-Wilk Normality Test Results:")
print("-" * 50)
normality_results = []
for emotion in emotion_cols:
    stat, p_value = stats.shapiro(emotions_mean_by_ida[emotion])
    normality_results.append({
        'emotion': emotion,
        'statistic': stat,
        'p_value': p_value,
        'normal': p_value > 0.05
    })
    
normality_df = pd.DataFrame(normality_results)
display(normality_df)

# Levene's test for homogeneity of variance
print("\nLevene's Test Results:")
print("-" * 50)
levene_results = []

# For Levene's test, we'll compare variances between all emotion columns
for i, emotion1 in enumerate(emotion_cols):
    for emotion2 in emotion_cols[i+1:]:
        stat, p_value = stats.levene(emotions_mean_by_ida[emotion1], 
                                   emotions_mean_by_ida[emotion2])
        levene_results.append({
            'emotion_pair': f"{emotion1} vs {emotion2}",
            'statistic': stat,
            'p_value': p_value,
            'equal_variance': p_value > 0.05
        })

levene_df = pd.DataFrame(levene_results)
display(levene_df)



Shapiro-Wilk Normality Test Results:
--------------------------------------------------


,emotion,statistic,p_value,normal
0,afraid_scared,0.890655,2.321193e-10,False
1,angry_frustrated,0.906893,2.280893e-09,False
2,anxious,0.919251,1.545992e-08,False
3,guilty,0.852547,2.326560e-12,False
4,happy,0.965439,1.641860e-04,False
5,hopeless,0.897605,5.998650e-10,False
6,indifferent,0.866648,1.150803e-11,False
7,inspired,0.972996,1.236958e-03,False
8,interested,0.940360,6.429837e-07,False
9,numb,0.718963,2.359322e-17,False



Levene's Test Results:
--------------------------------------------------


,emotion_pair,statistic,p_value,equal_variance
0,afraid_scared vs angry_frustrated,1.066515,0.302416,True
1,afraid_scared vs anxious,1.546899,0.214390,True
2,afraid_scared vs guilty,0.113111,0.736822,True
3,afraid_scared vs happy,4.382248,0.037003,False
4,afraid_scared vs hopeless,0.041820,0.838077,True
...,...,...,...,...
73,numb vs sad,6.925947,0.008855,False
74,numb vs stressed,10.997490,0.001004,False
75,optimistic_hopeful vs sad,1.477167,0.225002,True
76,optimistic_hopeful vs stressed,0.027374,0.868681,True


In [4]:
# Run both Pearson and Spearman correlations to compare age and average emotions

#merge age into same dataframe
emotions_with_age = emotions_mean_by_ida.merge(
    full_analysis_df[['ida', 'age']], 
    on='ida', 
    how='left'
)

#loop through all emotions and run correlation
results = []
for emotion in emotion_cols:
    pearson = stats.pearsonr(emotions_with_age['age'], emotions_with_age[emotion])
    spearman = stats.spearmanr(emotions_with_age['age'], emotions_with_age[emotion])

    results.append({
        'emotion': emotion,
        'pearson_r': pearson[0],
        'pearson_p': pearson[1],
        'spearman_r': spearman[0],
        'spearman_p': spearman[1]
    })

results_df = pd.DataFrame(results)
display(results_df)   

,emotion,pearson_r,pearson_p,spearman_r,spearman_p
0,afraid_scared,NaN,NaN,NaN,NaN
1,angry_frustrated,NaN,NaN,NaN,NaN
2,anxious,NaN,NaN,NaN,NaN
3,guilty,NaN,NaN,NaN,NaN
4,happy,NaN,NaN,NaN,NaN
5,hopeless,NaN,NaN,NaN,NaN
6,indifferent,NaN,NaN,NaN,NaN
7,inspired,NaN,NaN,NaN,NaN
8,interested,NaN,NaN,NaN,NaN
9,numb,NaN,NaN,NaN,NaN


In [5]:
# Mann-Whitney U test to compare emotions between genders
# ****** No significant results *****
print("\nMann-Whitney U Test Results for Gender Differences:")
print("-" * 50)

mw_results = []
for emotion in emotion_cols:
    # Get emotion scores for each gender
    male_scores = emotions_mean_by_ida[emotion][full_analysis_df['gender'] == 'Male']
    female_scores = emotions_mean_by_ida[emotion][full_analysis_df['gender'] == 'Female']
    
    # Perform Mann-Whitney U test
    stat, p_value = stats.mannwhitneyu(male_scores, female_scores, alternative='two-sided')
    
    mw_results.append({
        'emotion': emotion,
        'statistic': stat,
        'p_value': p_value,
        'significant': p_value < 0.05
    })

mw_results_df = pd.DataFrame(mw_results)
display(mw_results_df)



Mann-Whitney U Test Results for Gender Differences:
--------------------------------------------------


,emotion,statistic,p_value,significant
0,afraid_scared,4638.5,0.086955,False
1,angry_frustrated,4694.5,0.060536,False
2,anxious,4341.0,0.397472,False
3,guilty,3922.5,0.713011,False
4,happy,4050.5,0.995402,False
5,hopeless,4738.5,0.045168,True
6,indifferent,4687.5,0.062777,False
7,inspired,4038.0,0.978220,False
8,interested,4414.0,0.290770,False
9,numb,4165.5,0.716153,False


In [ ]:
#run spearman test on sassy vs average emots?